# Lab CMBLenses

## Model

In [1]:
] activate "/home/jaimerz/julia_repos/CMBLensingInferenceTestProblem"

  Activating project at `~/julia_repos/CMBLensingInferenceTestProblem`


In [2]:
using CMBLensing, CMBLensingInferenceTestProblem, CUDA, LaTeXStrings, MCMCDiagnosticTools, Plots, ProgressMeter, Random, Zygote
ENV["LINES"] = 10;

In [41]:
using Revise
using MCHMC

In [42]:
prob = load_cmb_lensing_problem(storage = Array,
                                T = Float32,
                                Nside = 16);

MAP_joint: 100%|████████████████████████████████████████| Time: 0:00:14
  step:        20
  logpdf:      6120.72
  α:           0.53812873
  CG:          2 iterations
  Linesearch:  12 bisections


In [43]:
target = CMBLensingTarget(prob)

CMBLensingTarget(770, MCHMC.var"#nlogp#25"{CMBLensingInferenceTestProblem.CMBLensingLogDensityProblem}(CMBLensingInferenceTestProblem.CMBLensingLogDensityProblem(BaseDataSet(d, Cf, Cn, Cn̂, M, M̂, B, B̂, logprior, Cϕ, Cf̃, D, G, L, Nϕ), [-9.376073837280273, -6.5940842628479, -5.405848979949951, -7.392019748687744, -8.028088569641113, -5.530062198638916, -1.2073196172714233, 1.3572702407836914, 3.625626564025879, 5.442736625671387  …  3.239753709749493e-8, -1.2966495432920055e-8, -8.702781428837625e-8, -1.6196999297335424e-7, -2.175492284095526e-7, -2.6352793724981893e-7, -3.134788357783691e-7, -3.52711253981397e-7, -1.6094379124341003, 0.0], [-9.446122169494629, -8.039124488830566, -5.012213706970215, -7.389093399047852, -7.862917423248291, -5.0858025550842285, -0.8858788013458252, 0.6133164763450623, 3.5648891925811768, 4.927055835723877  …  7.88092791026429e-9, -9.357064101322976e-8, -1.9338500578669482e-7, -2.2331580851187027e-7, -2.4733523673603486e-7, -3.1029253477754537e-7, -3.80

## Sample

In [44]:
eps=0.005
L=sqrt(2)
spl=Sampler(;eps=eps, L=L)

Sampler(Settings(MersenneTwister(0), 0.005, 1.4142135623730951, 0.0, 0.1931833275037836, 0.0005, 2000, 1000, 10, "LF"), MCHMC.Leapfrog)

In [45]:
init = MCHMC.Get_initial_conditions(spl, target)
x, u, g, r, t = init

([-41.040679931640625, -29.608135223388672, -24.1494197845459, -32.17665481567383, -34.82468032836914, -23.36894989013672, -4.4618940353393555, 7.117966651916504, 17.211938858032227, 24.473644256591797  …  -8.603252e6, 4.5899675e6, -7.408607e6, -1.9874535e6, 1.131673e7, 1.555073625e6, 616857.0, -1.945586e7, -1.6094379124341003, 0.0], [0.023955751787449433, 0.029222575122723263, -0.012452459414408416, -0.004757017460995747, 0.020693122318250313, 0.010488626051459846, 0.002291047472871313, -0.003845626358084955, -0.018138951242570353, 0.055535048076314056  …  0.030051469874635667, -0.02279153724520476, 0.044162865702707624, 0.07603616669611427, 0.056844528786071484, -0.021477145624383187, -0.01016862499294224, 0.008086194708853222, 0.011856808957284836, -0.02025110684626659], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, Inf, Inf], NaN, 0.0)

In [93]:
typeof(prob.Λmass)

Diagonal{ComplexF64, Field-(f°,ϕ°,θ)-Tuple{LambertEBFourier{Array{Float32, 3}}, LambertFourier{Array{ComplexF32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}}

In [91]:
simulate(Diagonal(one(LenseBasis(diag(prob.Λmass)))))

770-element Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float32, 3}}, LambertMap{Array{Float32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}:
  0.9160535335540771
  0.16928943991661072
  1.612758755683899
  ⋮
  0.42166241515579017
 -1.0350563424864263

In [81]:
u_test = similar(x)
u_test ./= u_test

770-element Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float32, 3}}, LambertMap{Array{Float32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}:
   1.0
   1.0
 NaN
   ⋮
   1.0
   1.0

In [85]:
similar(x)

770-element Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float32, 3}}, LambertMap{Array{Float32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}:
 -1.3519767463963716e37
  4.584908445424369e-41
  2.802596928649634e-45
  ⋮
  0.0
  0.0

In [52]:
convert(Vector{Float64}, g)

770-element Vector{Float64}:
  0.0
  0.0
  0.0
  ⋮
 Inf
 Inf

In [9]:
MCHMC.Random_unit_vector(spl, target)

3074-element Vector{Float64}:
 -0.0066933956091346685
  0.01689867709378862
 -0.0021468927074962483
  ⋮
 -0.01986356552740218
 -0.02533789071486329

In [30]:
function Update_momentum2(sampler, target, eff_eps, g, u, r)
    # TO DO: type inputs
    # Have to figure out where and when to define target
    """The momentum updating map of the ESH dynamics (see https://arxiv.org/pdf/2111.02434.pdf)"""

    g_norm = sqrt.(sum(g .^2 ))
    e = - g ./ g_norm
    println(length(u), typeof(u))
    println(length(e), typeof(e))
    ue = u .* vec(e) #dot(u, e)
    println("Hello")
    sh = sinh.(eff_eps * g_norm ./ target.d)
    ch = cosh.(eff_eps * g_norm ./ target.d)
    th = tanh.(eff_eps * g_norm ./ target.d)
    delta_r = @.(log(ch) + log1p(ue * th))

    uu = @.((u + e * (sh + ue * (ch - 1))) / (ch + ue * sh))
    rr = r .+ delta_r
    return uu, rr
end

Update_momentum2 (generic function with 1 method)

In [31]:
Update_momentum2(spl, target, eps, g, u, r)

3074Vector{Float64}
3074Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float64, 3}}, LambertMap{Array{Float64, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}


LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 3074 and 32

In [12]:
samples = Sample(spl, target; num_steps=10000, initial_x=Ω);

LoadError: DimensionMismatch: x and y are of different lengths!

## Plotting

In [15]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

In [16]:
plot([exp(Ω.θ.r) for (Ω,) in chain], label=L"r", xlabel="step")
plot!([exp(Ω.θ.Aϕ) for (Ω,) in chain], label=L"A_\phi")

LoadError: UndefVarError: `chain` not defined

In [17]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [18]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [19]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [20]:

ps = map([(:ϕ°,:I,L"L",L"\phi^\circ"), (:f°,:E,"L\ell",L"E^\circ"), (:f°,:B,L"\ell",L"B^\circ")]) do (k1, k2, xlabel, title)
    plot(get_Cℓ(prob.Ωtrue[k1][k2]); label="true", xlabel, title)
    plot!(get_Cℓ(prob.Ωstart[k1][k2]); label="start", xlabel, title)
    plot!(get_Cℓ(chain[end][1][k1][k2]); label="last sample", xlabel, title)
end
plot(ps..., layout=(1,3), xscale=:log10, yscale=:log10, size=(1000,300), legend=:bottomleft)

LoadError: UndefVarError: `prob` not defined